In [5]:
from classes.Dataset_Basic import Dataset_Basic
from classes.Process_Stages import Process_Stages

from classes.Dataset_Settings import Dataset_Settings

from run_models.cosine_sililarity.classes.Dataset_Cosine import Dataset_Cosine
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

from run_models.embed_words.classes.Embed_Words import Embed_Words

from run_models.gensim.services.gensim_services import gensim_download, gensim_save, gensim_load
from word_embedding.models.classes.EmbeddingModel import EmbeddingModel

name_dataset = "conceptnet_cosine_similarity"

datasets = {}
datasets["neural_course"] = Dataset_Cosine(
    df_name="neural_course",
    model_name=name_dataset, # used for dir name inside data_saved

    language="english",

    datasets = {
        "emebed_words_conceptnet": Dataset_Settings(
            df=None,
            may_run_now=False,
            required=True,
            parquet=True
        ),
        name_dataset: Dataset_Settings(
            df=None,
            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="gensim"
        ),
    },

    columns_to_add = {name_dataset: {"cosine_similarity": []}},

)

# get dataset, process dataset, save dataset
datasets["neural_course"].get_dataset()
datasets["neural_course"].process_dataset()
datasets["neural_course"].add_columns()
datasets["neural_course"].save()


Found neural_course with type .parquet in (data_saved/emebed_words_conceptnet) and converted it to df
df found: True, name: emebed_words_conceptnet
File neural_course not found in (data_saved/conceptnet_cosine_similarity)
df found: False, name: conceptnet_cosine_similarity
No need to run emebed_words_conceptnet for neural_course
Running conceptnet_cosine_similarity for neural_course
basic processing starting for neural_course


  0%|          | 0/646 [00:00<?, ?it/s]/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 646/646 [00:00<00:00, 30564.94it/s]

no saving needed because basic emebed_words_conceptnet already done on neural_course
saving new conceptnet_cosine_similarity phase for: neural_course


# Regression testing

In [1]:
# run regression
from run_models.regressions.classses.Regression_Model import Regression_Model

# dataset
from classes.Dataset_Settings import Dataset_Settings

import pandas as pd

# import datasets
data = pd.read_parquet("data_saved/cosine_similarity_glove/texas.parquet")
data.head(1)

# splitting data
seed = 42

dataset = Dataset_Settings(
    df=data, 
    may_run_now=True, 
    required=True
)

dataset.split_datasets(seed=seed)


In [2]:
data.head(2)


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points,cosine_similarity
0,0,What is the role of a prototype program in pro...,131,"[-3.1750799482688308, 4.741948984563351, -0.22...","[-1.4908400028944016, 1.6709499955177307, 0.20...",4,5,science,texas,0.8,0.683614
1,1,What is the role of a prototype program in pro...,131,"[-1.4845619946718216, 4.378678020089865, 1.191...","[-1.4908400028944016, 1.6709499955177307, 0.20...",5,5,science,texas,1.0,0.695208


In [9]:
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression, Ridge

# classes
from performance_tracking.classes.Measurement_Settings import Measurement_Settings

# constants
from performance_tracking.constants import *

isotonic_regression = Regression_Model(
    
    # id what is being tracked
    embedding_seperated=True,
    embedding_model_name="glove",
    classfication_model_name="ridge_regression",
    dataset_name="texas",

    # duplicates handeling
    settings_performance_tacking=REPLACE,
    measurement_settings=Measurement_Settings(
        print_regression=True, 
        print_classification=True, 
        save_performance=True
    ),
    
    dataset=dataset,
    classification_model=Ridge,
    x_column="cosine_similarity",
    y_column="normalized_points"
    
)


In [15]:
isotonic_regression.train()


TypeError: 'Ridge' object is not callable

In [16]:
isotonic_regression.test()



-----
embedding_model_name: glove
classfication_model_name: ridge_regression
dataset_name: texas

*** regression performance ***
rmse: 5.130799212147239
pears_correlation: 0.4465758064577274

*** classification performance ***
accuracy: 0.3788546255506608
precision_weighted: 0.5409047623756303
recall_weighted: 0.3788546255506608
f1_weighted: 0.3478631014039235


/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
isotonic_regression.validation()



-----
embedding_model_name: glove
classfication_model_name: ridge_regression
dataset_name: texas

*** regression performance ***
rmse: 4.787277886955549
pears_correlation: 0.35973980928430077

*** classification performance ***
accuracy: 0.42105263157894735
precision_weighted: 0.5791167516813711
recall_weighted: 0.42105263157894735
f1_weighted: 0.3875085008536521


/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
isotonic_regression.model_accuracy(True, "test_df")


Accuracy: 0.3788546255506608

Weighted-Averaged Precision: 0.5409047623756303
Weighted-Averaged Recall: 0.3788546255506608
Weighted-Averaged F1-score: 0.3478631014039235


/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
isotonic_regression["performance"]["rmse"]
# isotonic_regression["dataset"]["test_df"].head(20)


1.0261598424294478

In [9]:
dataset["test_df"].describe()


,row_id,question_id,assigned_points,max_points,normalized_points,cosine_similarity,y_pred,pred_points
count,454.000000,454.000000,454.000000,454.0,454.000000,454.000000,454.000000,454.000000
mean,1123.222467,170.002203,4.145374,5.0,0.829075,0.707738,0.824312,4.061674
std,659.512420,23.833353,1.145492,0.0,0.229098,0.197314,0.089210,0.510968
min,8.000000,131.000000,0.000000,5.0,0.000000,-0.004992,0.502070,3.000000
25%,535.750000,148.250000,4.000000,5.0,0.800000,0.616904,0.783244,4.000000
50%,1162.000000,171.000000,5.000000,5.0,1.000000,0.757160,0.846657,4.000000
75%,1705.250000,192.000000,5.000000,5.0,1.000000,0.847797,0.887636,4.000000
max,2271.000000,211.000000,5.000000,5.0,1.000000,1.000000,0.956451,5.000000


In [21]:
# model.model.index_to_key


In [22]:
# model.model[f"/c/en/open"]


In [18]:
performance = pd.read_csv("performance_tracking/tracking/performance_tracking.csv")

In [19]:
performance


,row_id,embedding_seperated,embedding_model_name,classification_model_name,dataset_name,dataset_split,time_stamp,rmse,pears_correlation,p_valueaccuracy,...,recall_macro,f1_macro,precision_micro,recall_micro,f1_micro,precision_weighted,recall_weighted,f1_weighted,p_value,accuracy
0,3,True,glove,ridge_regression,texas,train_df,2023-05-19 11:44:19,5.136838,0.391267,NaN,...,0.196450,0.162187,0.382150,0.382150,0.382150,0.597513,0.382150,0.343625,2.367019e-59,0.382150
1,4,True,glove,ridge_regression,texas,test_df,2023-05-19 11:44:19,5.130799,0.446576,NaN,...,0.202706,0.161333,0.378855,0.378855,0.378855,0.540905,0.378855,0.347863,1.226667e-23,0.378855
2,5,True,glove,ridge_regression,texas,validation_df,2023-05-19 11:44:19,4.787278,0.359740,NaN,...,0.212682,0.175319,0.421053,0.421053,0.421053,0.579117,0.421053,0.387509,2.265781e-08,0.421053


In [5]:
def get_yes_no_input(prompt):
    while True:
        user_input = input(prompt).lower().strip()
        if user_input in ('yes', 'y', 'no', 'n'):
            return user_input in ('yes', 'y')
        else:
            print("Invalid input. Please enter 'Yes' or 'No'.")

# Get user input
user_response = get_yes_no_input("Do you want to continue? (Yes/No): ")

if user_response:
    print("User chose Yes.")
else:
    print("User chose No.")


User chose Yes.
